## 作業目標: 使用Jieba進行各項的斷詞操作

這份作業我們會使用中文飯店評論資料集來作為斷詞練習。 [資料集:ChnSentiCorp_htl_all](https://github.com/SophonPlus/ChineseNlpCorpus)

---
### 讀入文本資料

In [3]:
import pandas as pd

# hint: 可利用pandas讀取CSV
pd_corpus = pd.read_csv('ChnSentiCorp_htl_all.csv')

pd_corpus.head(5)

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


In [4]:
#確認所有留言,正評價(label=1)與負評價數量(label=0)

pd_positive = pd_corpus[pd_corpus['label']==1]
pd_negative = pd_corpus[pd_corpus['label']==0]
print(f'Total: {len(pd_corpus)}, Positive: {len(pd_positive)}, Negative: {len(pd_negative)}')

Total: 7766, Positive: 5322, Negative: 2444


In [5]:
#檢查缺失值數量
print(pd_corpus.isna().sum())

# 1) 移除缺失值
pd_corpus = pd_corpus[pd_corpus.any(axis=1)==True]
# 2) 移除缺失值 (解答)
pd_corpus.dropna(inplace=True)

print(pd_corpus.isna().sum())

label     0
review    1
dtype: int64
label     0
review    0
dtype: int64


In [6]:
pd_positive = pd_corpus[pd_corpus['label']==1]
pd_negative = pd_corpus[pd_corpus['label']==0]
print(f'Total: {len(pd_corpus)}, Positive: {len(pd_positive)}, Negative: {len(pd_negative)}')

Total: 7765, Positive: 5322, Negative: 2443


---
### 建構結巴斷詞Function

建構可將讀入的Pandas DataFrame的文本資料，外加一欄名為`cut`的review欄位斷詞結果

In [7]:
import jieba

In [8]:
class JiebaCutingClass(object):
    '''Class to use jeiba to parse corpus from dataframe and cut the corpus
    
    Parameters
    -----------
    key_to_cut: str
        the dataframe key to parse the sentence for jieba cutting
    dic: str
        the dictionary to use for jieba, default is None (use default dictionary)
    userdict: str
        the user defined dictionary to use for jieba, default is None
    '''
    
    def __init__(self, key_to_cut:str, dic:str=None, userdict:str=None):
        
        if dic is not None:
            jieba.set_dictionary(dic)
        
        if userdict is not None:
            jieba.load_userdict(userdict)
        
        self.key_to_cut = key_to_cut
        
        #將paddle 功能開啟
        jieba.enable_paddle()
        
    @staticmethod
    def cut_single_sentence(sentence, use_paddle=False, use_full=False, use_search=False):
        
        if use_search:
            # hint:使用收尋引擎模式進行斷詞
            out = jieba.cut_for_search(sentence)
        else:
            # hint:非收尋引擎模式斷詞，請注意有精確模式、全模式與paddle模式
            out = jieba.cut(sentence, cut_all=use_full, use_paddle=use_paddle)
        return out
            
    
    def cut_corpus(self, corpus: pd.DataFrame, mode: str) -> pd.DataFrame:
        '''Method to read and cut sentence from dataframe and append another column named cut
        
        Paremeters
        --------------
        corpus: pd.DataFrame
            Input corpus in dataframe
        mode: str
            Jieba mode to be used
        
        Return
        ----------------
        out: pd.Dataframe
            Output corpus in dataframe
        '''
        
        # checking valid mode
        if mode not in ['paddle', 'full', 'precise', 'search']:
            raise TypeError(f'only support `paddle`, `full`, `precise`, and `search` mode, but get {mode}')
            
        # cut the corpus based on mode
        # hint: 根據mode來呼叫相對應的斷詞method
        if mode == 'paddle':
            out = self._paddle_cut(corpus)
        elif mode == 'full':
            out = self._full_cut(corpus)
        elif mode == 'precise':
            out = self._precise_cut(corpus)
        elif mode == 'search':
            out = self._search_cut(corpus)

        return out
    
    def _paddle_cut(self, corpus):
        '''paddle mode
        '''
        #enable paddle
        #hint:先啟用paddle mode
        jieba.enable_paddle()
        out = []
        # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
        for single_review in corpus[self.key_to_cut]:
            seg_list = JiebaCutingClass.cut_single_sentence(single_review, use_paddle=True)
            out.append(seg_list)
        corpus['cut'] = out
        
        return corpus
    
    def _full_cut(self, corpus):
        '''full mode
        '''
        
        out = []
        # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
        for single_review in corpus[self.key_to_cut]:
            seg_list = JiebaCutingClass.cut_single_sentence(single_review, use_full=True)
            out.append(word for word in seg_list)
            
        corpus['cut'] = out
        
        return corpus
    
    def _precise_cut(self, corpus):
        '''precise mode
        '''
        
        out = []
        # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
        for single_review in corpus[self.key_to_cut]:
            seg_list = [word for word in JiebaCutingClass.cut_single_sentence(single_review, use_full=False)]
            out.append(seg_list)
        corpus['cut'] = out
        
        return corpus
    
    def _search_cut(self, corpus):
        '''search mode
        '''
        
        out = []
        # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
        for single_review in corpus[self.key_to_cut]:
            seg_list = JiebaCutingClass.cut_single_sentence(single_review, use_search=True)
            out.append(word for word in seg_list)
        corpus['cut'] = out
        
        return corpus

---
### 使用建構好的斷詞物件對文本進行斷詞

In [9]:
### 請使用精確模式與預設字典對文本進行斷詞

# hint:請先實例化 JiebaCutingClass，再進行斷詞
jieba_cuting = JiebaCutingClass('review', dic='./dict.txt.big.txt', userdict='./userdict.txt')

pd_cut = jieba_cuting.cut_corpus(corpus=pd_corpus, mode='precise')
pd_cut.head(100)

Building prefix dict from C:\Users\jiaping.chu\OneDrive - 我傳媒科技股份有限公司\python\nlp_marathon\dict.txt.big.txt ...
Loading model from cache C:\Users\jiaping.chu\AppData\Local\Temp\jieba.uf2dab545c01dd14a591fd420a400dd97.cache
Loading model cost 1.214 seconds.
Prefix dict has been built successfully.
Paddle enabled successfully......


,label,review,cut
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较...","[距离, 川沙, 公路, 较近, ,, 但是, 公交, 指示, 不, 对, ,, 如果, 是..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!,"[商务, 大床, 房, ，, 房间, 很大, ，, 床有, 2M, 宽, ，, 整体, 感觉..."
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。,"[早餐, 太差, ，, 无论, 去, 多少, 人, ，, 那边, 也, 不加, 食品, 的,..."
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...,"[宾馆, 在, 小, 街道, 上, ，, 不大好, 找, ，, 但, 还好, 北京, 热心,..."
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风","[CBD, 中心, ,, 周围, 没什么, 店铺, ,, 说, 5, 星, 有点, 勉强, ..."
...,...,...,...
95,1,酒店非常棒，，环境一流，。。太美了，。。空气，绿化，。人流量。。温泉。。有机会可以再去。。。...,"[酒店, 非常, 棒, ，, ，, 环境, 一流, ，, 。, 。, 太美, 了, ，, 。..."
96,1,吸取网友们的经验，我们预定的是度假区高层海景大床房，于6月21-23日入住。很高兴亚太的服务...,"[吸取, 网友, 们, 的, 经验, ，, 我们, 预定, 的, 是, 度假区, 高层, 海..."
97,1,酒店的基本设施一般，但服务态度确实很不错，房间8楼以下就是新装修的，8楼的房间就比较成旧，洗...,"[酒店, 的, 基本, 设施, 一般, ，, 但, 服务态度, 确实, 很, 不错, ，, ..."
98,1,地理位置还可以，在西湖边上，附近有一些大排挡，饮食也比较方便。酒店服务很好，服务生和前台人员...,"[地理位置, 还, 可以, ，, 在, 西湖, 边上, ，, 附近, 有, 一些, 大排挡,..."


---
### 載入繁中字典為指定的字詞進行斷詞

In [10]:
test_string = '我愛cupoy自然語言處理馬拉松課程'
jieba_cuting = JiebaCutingClass('review', dic='./dict.txt.big.txt', userdict='userdict.txt')


out_string = jieba_cuting.cut_single_sentence(test_string, use_paddle=True) #paddle 模式
print(f'Paddle模式: {[string for string in out_string]}')

out_string = jieba_cuting.cut_single_sentence(test_string, use_full=True) #全模式
print(f'全模式: {[string for string in out_string]}')

out_string = jieba_cuting.cut_single_sentence(test_string, use_search=False) #搜尋模式
print(f'搜尋模式: {[string for string in out_string]}')

out_string = jieba_cuting.cut_single_sentence(test_string, use_full=False) #精確模式
print(f'精確模式: {[string for string in out_string]}')

Building prefix dict from C:\Users\jiaping.chu\OneDrive - 我傳媒科技股份有限公司\python\nlp_marathon\dict.txt.big.txt ...
Loading model from cache C:\Users\jiaping.chu\AppData\Local\Temp\jieba.uf2dab545c01dd14a591fd420a400dd97.cache
Loading model cost 1.207 seconds.
Prefix dict has been built successfully.
Paddle enabled successfully......


Paddle模式: ['我', '愛', 'cupoy', '自然', '語言', '處理', '馬拉松', '課程']
全模式: ['我', '愛', 'cupoy', '自然', '自然語言', '語言', '處理', '馬拉', '馬拉松', '課程']
搜尋模式: ['我', '愛', 'cupoy', '自然語言', '處理', '馬拉松', '課程']
精確模式: ['我', '愛', 'cupoy', '自然語言', '處理', '馬拉松', '課程']
